In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
import cv2
from PIL import Image
import matplotlib.pyplot as plt

import seaborn as sns
from mpl_toolkits.axes_grid1 import ImageGrid

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input, GlobalAveragePooling2D, BatchNormalization
)
from tensorflow.keras.activations import softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

In [ ]:
our_folders = ['Acne and Rosacea Photos', \
              'Herpes HPV and other STDs Photos', \
              'Atopic Dermatitis Photos']

In [ ]:
root_dir = '../input/dermnet/train'
test_dir = '../input/dermnet/test'

In [ ]:
acne_train_path = os.path.join(root_dir,'Acne and Rosacea Photos')
herpes_train_path = os.path.join(root_dir, 'Herpes HPV and other STDs Photos')
atopic_train_path = os.path.join(root_dir, 'Atopic Dermatitis Photos')

In [ ]:
acne_test_path = os.path.join(test_dir,'Acne and Rosacea Photos')
herpes_test_path = os.path.join(test_dir, 'Herpes HPV and other STDs Photos')
atopic_test_path = os.path.join(test_dir, 'Atopic Dermatitis Photos')

acne_test_files = ([files_ for _, _, files_ in os.walk(acne_test_path)])[0]
herpes_test_files = ([files_ for _, _, files_ in os.walk(herpes_test_path)])[0]
atopic_test_files = ([files_ for _, _, files_ in os.walk(atopic_test_path)])[0]

In [ ]:
train_dirs = []
for i in our_folders:
    for folder_,_, files_ in os.walk(f'../input/dermnet/train/{i}'):
        train_dirs.append(folder_)

In [ ]:
acne_train_files = ([files_ for _, _, files_ in os.walk(acne_train_path)])[0]
herpes_train_files = ([files_ for _, _, files_ in os.walk(herpes_train_path)])[0]
atopic_train_files = ([files_ for _, _, files_ in os.walk(atopic_train_path)])[0]

In [ ]:
def plotGridImages(d_name, list_files, train_path,nrows= 1, ncols=5):
    # for folder_name in our_folders:
    fig = plt.figure(1, figsize=(30, 30))
    grid = ImageGrid(fig, 111, nrows_ncols=(nrows, ncols), axes_pad=0.05)
    print(f"{d_name}")
    for i, img_id in enumerate(random.sample(list_files,ncols)):
        ax = grid[i]
        image_dir_path = os.path.join(train_path, img_id)
        img = image.load_img(image_dir_path, target_size=(224, 224), color_mode="rgb")
        img = image.img_to_array(img)
        ax.imshow(img / 255.)
        ax.text(10, 200, 'LABEL: %s' % d_name, color='k', backgroundcolor='w',\
        alpha=0.8)
        ax.axis('off')
    # plt.tight_layout()
    plt.show()

In [ ]:
plotGridImages('acne',acne_train_files, acne_train_path,ncols=5)

In [ ]:
plotGridImages('herpes',herpes_train_files, herpes_train_path,ncols=5)

In [ ]:
plotGridImages('atopic',atopic_train_files, atopic_train_path,ncols=5)

In [ ]:
plotGridImages('atopic',atopic_train_files, atopic_train_path,ncols=1)

In [ ]:
final_df = pd.DataFrame()

In [ ]:
acne_df = pd.DataFrame()
acne_df['Image'] = [acne_train_path+'/'+img for img in acne_train_files]
acne_df['Label'] = "acne"

In [ ]:
acne_df.shape

In [ ]:
herpes_df = pd.DataFrame()
herpes_df['Image'] =  [herpes_train_path+'/'+img for img in herpes_train_files]
herpes_df['Label'] = "herpes"

In [ ]:
herpes_df.shape

In [ ]:
atopic_df = pd.DataFrame()
atopic_df['Image'] =  [atopic_train_path+'/'+img for img in atopic_train_files]
atopic_df['Label'] = "atopic"

In [ ]:
atopic_df.shape

In [ ]:
final_df = final_df.append([acne_df, herpes_df, atopic_df])

In [ ]:
final_df.shape

In [ ]:
final_df.sample(10)

In [ ]:
ax = sns.countplot(x=final_df['Label'],
                   order=final_df['Label'].value_counts(ascending=False).index);

abs_values = final_df['Label'].value_counts(ascending=False).values

ax.bar_label(container=ax.containers[0], labels=abs_values);

In [ ]:
# Observation: Imabalnced dataset

In [ ]:
final_test_df = pd.DataFrame()
################# acne #########

acne_test_df = pd.DataFrame()
acne_test_df['Image'] = [acne_test_path+'/'+img for img in acne_test_files]
acne_test_df['Label'] = "acne"

##################  herpes  ##########

herpes_test_df = pd.DataFrame()
herpes_test_df['Image'] =  [herpes_test_path+'/'+img for img in herpes_test_files]
herpes_test_df['Label'] = "herpes"

################  atopic  ##########

atopic_test_df = pd.DataFrame()
atopic_test_df['Image'] =  [atopic_test_path+'/'+img for img in atopic_test_files]
atopic_test_df['Label'] = "atopic"
###########################################
###########################################

final_test_df = final_test_df.append([acne_test_df, 
                                      herpes_test_df, 
                                      atopic_test_df])

final_test_df.shape

In [ ]:
# applying image augmentation techniques

In [ ]:
train_data_gen  = ImageDataGenerator(
                                    rescale=1 / 255.0,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    horizontal_flip = True,
                                    vertical_flip = True,
                                    validation_split=0.2
                                    ,fill_mode='nearest')
test_data_gen = ImageDataGenerator(rescale=1 / 255.0)

In [ ]:
batch_size = 64
train_generator = train_data_gen.flow_from_dataframe(
    dataframe=final_df,
    x_col="Image",
    y_col="Label",
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode="categorical",#sparse
    subset='training',
    shuffle=True,
    seed=42
)
valid_generator = train_data_gen.flow_from_dataframe(
    dataframe=final_df,
    x_col="Image",
    y_col="Label",
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode="categorical", #sparse
    subset='validation',
    shuffle=True,
    seed=42 
)
test_generator = test_data_gen.flow_from_dataframe(
    dataframe=final_test_df,
    x_col="Image",
    y_col="Label",
    target_size=(256, 256),
    batch_size=1,
    class_mode='categorical',
    shuffle=False,
)

In [ ]:
# VGG16 with Input shape of our Images
# Include Top is set to false to allow us to add more layers

res = ResNet50(weights ='imagenet', include_top = False, 
               input_shape = (256, 256, 3)) 
               
# Setting the trainable to false
res.trainable = False


x= res.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
# x = Dropout(0.5)(x) 
x = Dense(512, activation ='relu')(x)
x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

x = Dense(256, activation ='relu')(x)
x = BatchNormalization()(x)

x = Dense(3, activation ='softmax')(x)
model = Model(res.input, x)

model.compile(optimizer =tf.keras.optimizers.RMSprop(learning_rate=0.0001),  #'Adam'
              loss ="categorical_crossentropy",  #sparse_categorical_crossentropy
              metrics =["categorical_accuracy"])  #sparse_categorical_accuracy

model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    min_delta=0.0001, 
    mode='min'
)

In [ ]:
history = model.fit(train_generator, 
                    epochs=100, 
                    validation_data=valid_generator)

Accuracy plot

In [ ]:
#plot accuracy vs epoch
plt.plot(history.history['categorical_accuracy']) #sparse_categorical_accuracy
plt.plot(history.history['val_categorical_accuracy']) #val_sparse_categorical_accuracy
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Loss plot

In [ ]:
# Plot loss values vs epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
test_true=test_generator.classes
test_pred_raw = model.predict(test_generator)
test_pred = np.argmax(test_pred_raw, axis=1)

cm = confusion_matrix(test_true, test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[acne_test_df, herpes_test_df, atopic_test_df])
fig, ax = plt.subplots(figsize=(15,15))
disp.plot(ax=ax,cmap=plt.cm.Blues)
plt.show()

In [ ]:
result = model.evaluate(test_generator,batch_size=64)
print("test_loss, test accuracy",result)

In [ ]:
vgg_preds = model.predict(test_generator)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [ ]:
true_classes = test_generator.classes
class_indices = train_generator.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())


vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
print("Resnet50 Model Accuracy: {:.2f}%".format(vgg_acc * 100))

In [ ]:
img = tf.keras.utils.load_img(
    '../input/dermnet/test/Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions/actinic-keratosis-5FU-32.jpg',
    target_size=(256, 256)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

In [ ]:
score = tf.nn.softmax(predictions[0])

class_names = list(train_generator.class_indices)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
def predict_image_class(image_path,true_value):

    img = cv2.imread(image_path)
    img = cv2.resize(img,(256,256))

    img = np.expand_dims(img, axis=0)

    img_class = model.predict(img, batch_size=1)
    
    score = tf.nn.softmax(img_class[0])

    class_names = list(train_generator.class_indices)

    print(
        "This image most likely belongs to {}"
        .format(class_names[np.argmax(score)])
    )
    
    
    # for folder_name in our_folders:
    fig = plt.figure(1, figsize=(15, 15))
    grid = ImageGrid(fig, 111, nrows_ncols=(1, 1), axes_pad=0.05)
    ax = grid[0]
    img = load_img(image_path, (256, 256), color_mode="rgb")
    img = img_to_array(img)
    ax.imshow(img / 255.)
    
    ax.text(10, 150, 'True Label: %s' % true_value.upper(), color='g', backgroundcolor='w',\
    alpha=0.8, size = 20)
    
    ax.text(10, 200, 'Predicted Label: %s' % class_names[np.argmax(score)].upper(), color='k', backgroundcolor='w',\
    alpha=0.8, size = 20)
    ax.axis('off')
    plt.show()

In [ ]:
predict_image_class('/kaggle/input/dermnet/test/Acne and Rosacea Photos/acne-closed-comedo-15.jpg','Acne')

In [ ]:
predict_image_class('/kaggle/input/dermnet/test/Herpes HPV and other STDs Photos/11herpesAnal0913041.jpg','Herpes')

In [ ]:
predict_image_class('/kaggle/input/dermnet/test/Atopic Dermatitis Photos/05Atopic12060414.jpg','Atopic')

In [ ]:
# Wrong predictions
predict_image_class('../input/dermnet/test/Acne and Rosacea Photos/Forest-2.jpg','unknown')